# < 24. 딥러닝 레이어의 이해 (2) Embedding, Recurrent > 

## 학습목표
1. 레이어의 개념을 이해한다.
2. 딥러닝 모델 속 각 레이어(Embedding ,RNN, LSTM)의 동작 방식을 이해한다
3. 데이터의 특성을 고려한 레이어를 설계하고, 이를 Tensorflow로 정의하는 법을 배운다

## 목차
1. 분포 가설과 분산 표현
2. 단어를 부탁해! Embedding 레이어
3. 순차적인 데이터! Recurrent 레이어 (1) RNN
4. 순차적인 데이터! Recurrent 레이어 (2) LSTM
5. 마무리

---

## 24.2 분포 가설과 분산 표현

<mark>희소표현(Sparse Representation)</mark>
- 벡터의 특정 차원에 단어 혹은 의미리르 직접 매핑하는 방식

### 단어의 분산 표현(Distributed Representation)

<mark>분포가설(distribution hypothesis)</mark> 
- 유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다 
- 맥락이란?
    - 맥락이라는 것은 단순하게 단어 좌우에 출현하는 다른 단어들이다
        - 나는 밥을 먹는다
        - 나는 떡을 먹는다
        - 나는 _을 먹는다
- 나는 과 먹는다 사이에 주로 나타나는 것들 사이에 어떤 의미적인 유사점이 있다. 그렇다고 컴퓨터에게 이 유사점이 무엇인지를 알려줄 방법은 없다. 우리는 지금 모든 단어를 고정 차원의 벡터로 표현하려고 한다 <br>

<mark>분산표현(Distributed Representation)</mark>
- 단어 벡터를 단어의 분산표현이라고 한다
- 분산 표현을 사용하면 희소 표현과는 다르게 단어 간의 유사도 를 계산으로 구할 수 있다는 장점이 있다

---

## 24.3 단어를 부탁해! Embedding 레이어

<mark>Embedding레이어</mark>
- 컴퓨터용 단어 사전
- 우리가 "단어를 n개 쓸 거야~" 라고 전달만 해주면 컴퓨터는 알아서 사전을 만들고, 수많은 데이터를 거치며 각 단어의 의미(분산 표현)를 차근차근 업데이트
![](https://d3s0tskafalll9.cloudfront.net/media/images/F-24-12.max-800x600.png)

- 입력으로 들어온 단어를 분산 표현으로 연결해 주는 역할을 하는데 그것이 Weight에서 특정 행을 읽어오는 것과 같아 이 레이어를 룩업 테이블(Lookup Table) 이라고 부르기도 한다

<mark>원-핫인코딩</mark>
- https://www.kakaobrain.com/blog/6

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


원-핫 벡터를 Linear 레이어에 넣는다

In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[ 0.72788644 -0.67224854]
 [-0.73388624 -0.3331143 ]
 [-0.06639606  0.16395164]
 [ 0.30684733 -0.33530533]
 [-0.06899005 -0.06330311]
 [ 0.6955668   0.46933687]
 [ 0.741989   -0.70360136]
 [-0.65201    -0.02520192]]

One-Hot Linear Result
[[ 0.72788644 -0.67224854]
 [ 0.72788644 -0.67224854]
 [ 0.72788644 -0.67224854]
 [ 0.72788644 -0.67224854]
 [-0.73388624 -0.3331143 ]
 [-0.06639606  0.16395164]
 [ 0.30684733 -0.33530533]
 [-0.06899005 -0.06330311]
 [-0.06899005 -0.06330311]
 [-0.06899005 -0.06330311]]


- 원-핫 벡터에 Linear 레이어를 적용하니 Linear 레이어의 Weight에서 단어 인덱스 배열 <code>[ 0, 0, 0, 0, 1, 2, 3, 4, 4, 4 ]</code> 에 해당하는 행만 읽어오는 효과가 있다!

![](https://d3s0tskafalll9.cloudfront.net/media/images/F-24-12.max-800x600.png)

- 각 단어를 원-핫 인코딩해서 Linear 연산을 하는 것이 바로 파란 선의 정체
- 원-핫 인코딩을 위한 단어 사전을 구축하고 단어를 사전의 인덱스로 변환만 해주면 Embedding 레이어를 완벽하게 사용할 수 있다

In [3]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산 표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


- Embedding 레이어는 입력에 직접 연결되게 사용해야 한다!!!

---

## 24.4 순차적인 데이터! Recurrent 레이어 (1) RNN

<mark>RNN</mark> 
- Recurrent Neural Network 
- 순차 데이터를 처리하기 위해 고안된 것
- 연속성을 다루는 레이어

![](https://miro.medium.com/max/480/1*TqcA9EIUF-DGGTBhIx_qbQ.gif)

- RNN의 입력으로 들어가는 모든 단어만큼 Weight를 만드는 게 아님에 유의
- <code>(입력의 차원, 출력의 차원)</code>에 해당하는 단 하나의 Weight를 순차적으로 업데이트하는 것이 RNN

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/F-24-13.png)

- 첫 입력인 What의 정보가 마지막 입력인 ?에 다다라서는 거의 희석된 모습을 보여주고 있다
- 입력의 앞부분이 뒤로 갈수록 옅어져 손실이 발생한다
- 이를 <a>기울기소실</a> 문제라고 한다

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


- 어떤 문장이 긍정인지 부정인지 나누기 위해서라면 문장을 모두 읽은 후, 최종 step의 Output만 확인해도 판단이 가능
- 문장을 생성하는 경우라면 이전 단어를 입력으로 받아 생성된 모든 다음 단어, 즉 ㅁ든 step에 대한 output이 필요
- <code>tf.keras.layers.SimpleRNN</code> 레이어의 <code>return_sequences</code> 인자를 조절함으로써 조절할 수 있

![](https://d3s0tskafalll9.cloudfront.net/media/images/F-24-14.max-800x600.png)

In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


---

## 24.5 순차적인 데이터! Recurrent 레이어 (2) LSTM

### <mark>LSTM</mark>
- Long Short-Term Memory의 약어로 기울기 소실 문제를 해결하기 위해 고안된 RNN 레이어
- RNN의 네트워크 구조 특성상, 입력되는 문장이 길수록 초기에 입력된 단어들의 미분 값이 매우 작아지거나 커지는 현상이 발생
- 만약 미분 값이 너무 작아진다면 이를 Vanishing Gradient라고 부르고, 너무 커진다면 Exploding Gradient라고 한다
- 전자의 경우, 가중치 업데이트가 잘 안되니 학습이 거의 이뤄지지 않는다. 후자의 경우, 가중치 업데이트가 너무 크니 학습이 불안정
- LSTM 은 기본적인 바닐라(Simple) RNN보다 4배나 큰 Weight를 가지고 있음
- 4배 깊은 RNN이라고 표현하기보다, 4종류의 서로 다른 Weight를 가진 RNN이라고 이해하시는 것이 좋다
- 각 Weight들은 Gate라는 구조에 포함되어 어떤 정보를 기억하고, 어떤 정보를 다음 스텝에 전달할지 등을 결정

- LSTM에는 Cell state 라는 새로운 개념이 추가되는데, 긴 문장이 들어와도 이 Cell state 를 통해 오래된 기억 또한 큰 손실 없이 저장


![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile30.uf.tistory.com%2Fimage%2F999F603E5ACB86A00550F0)

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F993A93495ACB86A02FFAA8)

### <a>LSTM의 핵심 아이디어</a>

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile10.uf.tistory.com%2Fimage%2F99CB87505ACB86A00FAB6F)

- LSTM의 핵심은 cell state인데, 모듈 그림에서 수평으로 그어진 윗 선에 해당
- Cell state는 컨베이어 벨트와 같아서, 작은 linear interaction만을 적용시키면서 전체 체인을 계속 구동시킨다. 정보가 전혀 바뀌지 않고 그대로 흐르게만 하는 것은 매우 쉽게 할 수 있다.


![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile29.uf.tistory.com%2Fimage%2F99C98C4F5ACB86A01FD4E5)

- Gate는 정보가 전달될 수 있는 추가적인 방법으로, sigmoid layer와 pointwise 곱셈으로 이루어져 있다.
- Sigmoid layer는 0과 1 사이의 숫자를 내보내는데, 이 값은 각 컴포넌트가 얼마나 정보를 전달해야 하는지에 대한 척도를 나타낸다. 그 값이 0이라면 "아무 것도 넘기지 말라"가 되고, 값이 1이라면 "모든 것을 넘겨드려라"가 된다.

- LSTM은 3개의 gate를 가지고 있고, 이 문들은 cell state를 보호하고 제어한다.


### <a>단계별로 보는 LSTM</a>

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile2.uf.tistory.com%2Fimage%2F9957DB445ACB86A02155EA)

#### 1.  forget gate layer
- cell state로부터 어떤 정보를 버릴 것인지를 정하는 것으로, sigmoid layer에 의해 결정된다. 그래서 이 단계의 gate를 "forget gate layer"라고 부른다. 이 단계에서는 $h_{t-1} $ 과 $x_{t} $ 를 받아서 0과 1 사이의 값을 $C_{t-1}$에 보내준다

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile4.uf.tistory.com%2Fimage%2F99D969495ACB86A00BFC15)

#### 2. input gate layer
- 다음 단계는 앞으로 들어오는 새로운 정보 중 어떤 것을 cell state에 저장할 것인지를 정한다
- "input gate layer"라고 불리는 sigmoid layer가 어떤 값을 업데이트할 지 정한다. 그 다음에 tanh layer가 새로운 후보 값들인 C_t라는 라는 vector를 만들고, cell state에 더할 준비를 한다. 이렇게 두 단계에서 나온 정보를 합쳐서 state를 업데이트할 재료를 만들게 된다.


![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile9.uf.tistory.com%2Fimage%2F997589405ACB86A00CADEA)

#### 3. cell state 업데이트
- 우선 이전 state에 f_t를 곱해서 가장 첫 단계에서 잊어버리기로 정했던 것들을 진짜로 잊어버린다. 그리고 나서 i_t * C_t 를 더한다 
- 이 더하는 값은 두 번째 단계에서 업데이트하기로 한 값을 얼마나 업데이트할 지 정한 만큼 scale한 값이 된다.
- 또 다시 언어 모델 문제로 돌아가보면, 이 단계에서 실제로 이전 주어의 성별 정보를 없애고 새로운 정보를 더하게 되는데, 이는 지난 단계들에서 다 정했던 것들을 실천만 하는 단계임을 다시 확인할 수 있다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile7.uf.tistory.com%2Fimage%2F99FB824C5ACB86A10D4182)

#### 4. output gate layer
- 마지막으로 무엇을 output으로 내보낼 지 정하는 일이 남았다. 이 output은 cell state를 바탕으로 필터된 값이 될 것이다. 
- 가장 먼저, sigmoid layer에 input 데이터를 태워서 cell state의 어느 부분을 output으로 내보낼 지를 정한다. 
- 그리고나서 cell state를 tanh layer에 태워서 -1과 1 사이의 값을 받은 뒤에 방금 전에 계산한 sigmoid gate의 output과 곱해준다. 그렇게 하면 우리가 output으로 보내고자 하는 부분만 내보낼 수 있게 된다.


출처 : https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr

### 양방향(Bidirectional) RNN

- 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태
![](https://d3s0tskafalll9.cloudfront.net/media/images/F-24v2-2.max-800x600.jpg)

In [6]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


- Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 우리가 앞에서 배운 RNN의 2배 크기 Weight가 정의